### DECAGON TEST

1. Create an ETL workflow that brings the data into an SQL database. 
    - You can use Postgres or MySQL. We plan to have a database that contains all **continents, countries, currencies, and languages. (continents, countries, currencies, languages should all be in a different table)**
    
    

2. With the data in the database, create the following reports using SQL.
    - List all the continents and the total number of countries in each. 
    For example, Africa 100, Europe 10, etc. The continent's name and country
    - List all the languages and commas separated countries that speak thelanguage.
    - List all the countries and the total number of languages spoken.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

### DATA SOURCES

In [2]:
# data sources

countries_json = 'https://raw.githubusercontent.com/annexare/Countries/master/data/countries.json'
continent_json = 'https://raw.githubusercontent.com/annexare/Countries/master/data/continents.json'
languages_json = 'https://raw.githubusercontent.com/annexare/Countries/master/data/languages.json'
countries_iso2_to_iso3_json = 'https://raw.githubusercontent.com/annexare/Countries/master/data/countries.2to3.json'

### EXTRACT & TRANSFORMATION OF DATA `E-T`

In [3]:
# Extracting and transforming Countries Table
def extract_and_transform_countries_data():
    countries_df = pd.read_json(countries_json)

    countries_df = countries_df.T

    countries_df.reset_index(inplace = True)

    countries_df.rename(columns = {'index':'iso2Code', 
                                   'native':'nativeName', 
                                   'currency':'currencyCode', 
                                   'languages':'languageCode',
                                   'phone':'phoneCode',
                                   'continent':'continentCode'}, inplace = True)
    
    countries_df['currencyCode'] = [','.join(i) if isinstance(i, list) else i for i in countries_df['currencyCode']]

    countries_df['languageCode'] = [','.join(i) if isinstance(i, list) else i for i in countries_df['languageCode']]

    countries_df['phoneCode'] = [i[0] for i in countries_df['phoneCode']]
    
    countries_df.drop(['continents'], axis=1, inplace = True)
    
    return countries_df
    
def extract_and_transform_countries_iso2_to_iso3_data():
    countries_iso2_to_iso3_df = pd.read_json(countries_iso2_to_iso3_json, typ='series')

    countries_iso2_to_iso3_df = countries_iso2_to_iso3_df.to_frame()

    countries_iso2_to_iso3_df.reset_index(inplace = True)

    countries_iso2_to_iso3_df.rename(columns = {'index':'iso2Code',0:'iso3Code' }, inplace = True)
    
    countries_new_df = extract_and_transform_countries_data()
    
    # append iso3Code to countries table from countries_iso2_to_iso3_df
    
    countries_new_df = pd.merge(countries_new_df, countries_iso2_to_iso3_df, on ='iso2Code')
    
    fields = ['name', 'capital','nativeName','iso2Code', 
           'iso3Code', 'phoneCode', 'continentCode','currencyCode','languageCode']
    
    countries_new_df = countries_new_df[fields]
    
    return countries_new_df


# Extracting and transforming continents Table
def extract_and_transform_continents_data():
    continents_df = pd.read_json(continent_json, typ='series')

    continents_df = continents_df.to_frame()

    continents_df.reset_index(inplace = True)

    continents_df.rename(columns = {'index':'code',0:'name' }, inplace = True)
    
    return continents_df


# Extracting and transforming Currency Table
def extract_and_transform_currency_data():
    
    countries_df = extract_and_transform_countries_iso2_to_iso3_data()
    
    currency_data = countries_df[['iso2Code','currencyCode']].copy()

    currency_data = currency_data.explode('currencyCode')

    currency_data = currency_data.explode('currencyCode')

    currency_data = currency_data.groupby('currencyCode')['iso2Code'].apply(list)

    currency_data = currency_data.to_frame()

    currency_data.reset_index(inplace = True)

    currency_data.rename(columns = {'currencyCode':'code','iso2Code':'countryIso2code' }, inplace = True)

    currency_data['countryIso2code'] = [','.join(i) if isinstance(i, list) else i for i in currency_data['countryIso2code']]
    
    return currency_data

def extract_and_transform_language_data():
    languages_df = pd.read_json(languages_json)

    languages_df = languages_df.T

    languages_df.reset_index(inplace = True)

    languages_df.rename(columns = {'index':'code', 'native':'nativeName'}, inplace = True)
    
    languages_df.drop('rtl',axis=1, inplace = True)

    return languages_df

In [20]:
extract_and_transform_countries_data()

,iso2Code,name,nativeName,phoneCode,continentCode,capital,currencyCode,languageCode
0,AD,Andorra,Andorra,376,EU,Andorra la Vella,EUR,ca
1,AE,United Arab Emirates,دولة الإمارات العربية المتحدة,971,AS,Abu Dhabi,AED,ar
2,AF,Afghanistan,افغانستان,93,AS,Kabul,AFN,"ps,uz,tk"
3,AG,Antigua and Barbuda,Antigua and Barbuda,1268,NA,Saint John's,XCD,en
4,AI,Anguilla,Anguilla,1264,NA,The Valley,XCD,en
...,...,...,...,...,...,...,...,...
245,YE,Yemen,اليَمَن,967,AS,Sana'a,YER,ar
246,YT,Mayotte,Mayotte,262,AF,Mamoudzou,EUR,fr
247,ZA,South Africa,South Africa,27,AF,Pretoria,ZAR,"af,en,nr,st,ss,tn,ts,ve,xh,zu"
248,ZM,Zambia,Zambia,260,AF,Lusaka,ZMW,en


In [5]:
# Visualize Countries Table
extract_and_transform_countries_iso2_to_iso3_data()

,name,capital,nativeName,iso2Code,iso3Code,phoneCode,continentCode,currencyCode,languageCode
0,Andorra,Andorra la Vella,Andorra,AD,AND,376,EU,EUR,ca
1,United Arab Emirates,Abu Dhabi,دولة الإمارات العربية المتحدة,AE,ARE,971,AS,AED,ar
2,Afghanistan,Kabul,افغانستان,AF,AFG,93,AS,AFN,"ps,uz,tk"
3,Antigua and Barbuda,Saint John's,Antigua and Barbuda,AG,ATG,1268,NA,XCD,en
4,Anguilla,The Valley,Anguilla,AI,AIA,1264,NA,XCD,en
...,...,...,...,...,...,...,...,...,...
245,Yemen,Sana'a,اليَمَن,YE,YEM,967,AS,YER,ar
246,Mayotte,Mamoudzou,Mayotte,YT,MYT,262,AF,EUR,fr
247,South Africa,Pretoria,South Africa,ZA,ZAF,27,AF,ZAR,"af,en,nr,st,ss,tn,ts,ve,xh,zu"
248,Zambia,Lusaka,Zambia,ZM,ZMB,260,AF,ZMW,en


In [6]:
# Visualize Continent Table
extract_and_transform_continents_data()

,code,name
0,AF,Africa
1,AN,Antarctica
2,AS,Asia
3,EU,Europe
4,NA,North America
5,OC,Oceania
6,SA,South America


In [7]:
# Visualize Language Table
extract_and_transform_language_data()

,code,name,nativeName
0,aa,Afar,Afar
1,ab,Abkhazian,Аҧсуа
2,af,Afrikaans,Afrikaans
3,ak,Akan,Akana
4,am,Amharic,አማርኛ
...,...,...,...
180,yi,Yiddish,ייִדיש
181,yo,Yoruba,Yorùbá
182,za,Zhuang,Cuengh / Tôô / 壮语
183,zh,Chinese,中文


In [8]:
# Visualize Currency Table
extract_and_transform_currency_data()

,code,countryIso2code
0,,AQ
1,AED,AE
2,AFN,AF
3,ALL,AL
4,AMD,AM
...,...,...
155,XOF,"BF,BJ,CI,GW,ML,NE,SN,TG"
156,XPF,"NC,PF,WF"
157,YER,YE
158,ZAR,ZA


### SCHEMA DEFINITION
- This script below was loaded via **`the terminal`**, using **```mysql -h localhost -u root```***

In [21]:
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode

DB_NAME = 'decagon_test_db'

TABLES = {}

TABLES['countries'] = (
  "CREATE TABLE IF NOT EXISTS `countries` ("
  "`name` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`capital` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`nativeName` varchar(255) COLLATE utf8mb4_unicode_ci NULL,"
  "`iso2Code` varchar(255) NOT NULL,"
  "`iso3Code` varchar(255) NOT NULL,"
  "`phoneCode` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`currencyCode` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`continentCode` varchar(255)COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`languageCode` varchar(255)COLLATE utf8mb4_unicode_ci NOT NULL,"
  "PRIMARY KEY (`name`)"
") ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")


TABLES['continents'] = (
    "CREATE TABLE IF NOT EXISTS `continents` ("
      "`code` varchar(255) NOT NULL,"
      "`name` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
      "PRIMARY KEY (`code`)"
    ") ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")


TABLES['currency'] = (
    "CREATE TABLE IF NOT EXISTS `currency` ("
      "`code` varchar(5) COLLATE utf8mb4_unicode_ci NOT NULL,"
      "`name` varchar(100) COLLATE utf8mb4_unicode_ci NULL,"
      "`description` varchar(255) COLLATE utf8mb4_unicode_ci DEFAULT NULL,"
      "`symbol` varchar(5) COLLATE utf8mb4_unicode_ci DEFAULT NULL,"
     " PRIMARY KEY (`code`)"
   " ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")


TABLES['languages'] = (
    "CREATE TABLE IF NOT EXISTS `languages` ("
      "`code` varchar(255) NOT NULL,"
      "`name` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
      "`nativeName` varchar(255) COLLATE utf8mb4_unicode_ci NULL,"
      "PRIMARY KEY (`code`)"
    ") ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")

### CREATE DATABASE (IF IT DOESN'T EXIST)

In [22]:
cnx = mysql.connector.connect(user='root')
cursor = cnx.cursor()

def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

Database decagon_test_db does not exists.
Database decagon_test_db created successfully.


### CREATE TABLES (IF THEY DON'T EXIST)

In [23]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

# cursor.close()
# cnx.close()

Creating table countries: OK
Creating table continents: OK
Creating table currency: OK
Creating table languages: OK


### LOAD DATA INTO TABLES `L`

In [24]:
countries = [tuple(r) for r in extract_and_transform_countries_iso2_to_iso3_data().to_numpy()]
currency = [tuple(r) for r in extract_and_transform_currency_data().to_numpy()]
languages = [tuple(r) for r in extract_and_transform_language_data().to_numpy()]
continents = [tuple(r) for r in extract_and_transform_continents_data().to_numpy()]

In [25]:
try:
    
    cnx = mysql.connector.connect(user='root')
    
    cursor = cnx.cursor()

    cursor.execute("USE {}".format(DB_NAME))
    
    countries_insert_query = """INSERT INTO countries (name, capital, nativeName,iso2Code, iso3Code, phoneCode, continentCode, currencyCode, languageCode) 
                               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) """

    countries_records_to_insert = countries

    #     cursor = connection.cursor()
    cursor.executemany(countries_insert_query, countries_records_to_insert)
    cnx.commit()
    print(cursor.rowcount, "Record inserted successfully into countries table")

except mysql.connector.Error as error:
    print("Failed to insert countries record into MySQL countries table {}".format(error))

    
try:
        
    continents_insert_query = """INSERT INTO continents (code, name) 
                               VALUES (%s, %s) """

    continents_records_to_insert = continents

    #     cursor = connection.cursor()
    cursor.executemany(continents_insert_query, continents_records_to_insert)
    cnx.commit()
    print(cursor.rowcount, "Record inserted successfully into continents table")

except mysql.connector.Error as error:
    print("Failed to insert continents record into MySQL continents table {}".format(error))   

    
try:
        
    languages_insert_query = """INSERT INTO languages (code, name, nativeName) 
                               VALUES (%s, %s, %s) """

    languages_records_to_insert = languages

    #     cursor = connection.cursor()
    cursor.executemany(languages_insert_query, languages_records_to_insert)
    cnx.commit()
    print(cursor.rowcount, "Record inserted successfully into languages table")

except mysql.connector.Error as error:
    print("Failed to insert languages record into MySQL languages table {}".format(error))     



250 Record inserted successfully into countries table
7 Record inserted successfully into continents table
185 Record inserted successfully into languages table


#### ANSWER TO QUESTION 1
- List all the continents and the total number of countries in each

In [14]:
QUERY = """select a.name as Continent, count(b.name) as 'Country Count' from continents a left join countries b on a.code = b.continentCode group by a.name"""

cursor.execute(QUERY)
a = cursor.fetchall()

ANSWER1 = pd.DataFrame(a, columns =['Continent', 'Country Count'])
ANSWER1

,Continent,Country Count
0,Africa,58
1,Antarctica,5
2,Asia,52
3,Europe,53
4,North America,41
5,Oceania,27
6,South America,14


#### ANSWER TO QUESTION 2
- List all the languages and commas separated countries that speak thelanguage.

In [28]:
QUERY = """
            with language_country_data as (
                SELECT
                  countries.name,
                  SUBSTRING_INDEX(SUBSTRING_INDEX(countries.languageCode, ',', numbers.n), ',', -1) languageCode
                FROM
                  (SELECT 1 n UNION ALL SELECT 2
                   UNION ALL SELECT 3 UNION ALL SELECT 4) numbers INNER JOIN countries
                  ON CHAR_LENGTH(countries.languageCode)
                     -CHAR_LENGTH(REPLACE(countries.languageCode, ',', ''))>=numbers.n-1
                ORDER BY
                  name, n)

                select 
                    b.name as 'Language', 
                    group_concat(a.name) as Countries
                from language_country_data a 
                left join languages b on a.languageCode = b.code
                group by b.name

        """

cursor.execute(QUERY)
a = cursor.fetchall()

ANSWER2 = pd.DataFrame(a, columns =['Language', 'Countries'])
ANSWER2

,Language,Countries
0,None,Antarctica
1,Afrikaans,"Namibia,South Africa"
2,Albanian,"Albania,Kosovo,Montenegro"
3,Amharic,Ethiopia
4,Arabic,"Algeria,Bahrain,Chad,Comoros,Djibouti,Egypt,Er..."
...,...,...
104,Turkmen,"Turkmenistan,Afghanistan"
105,Ukrainian,Ukraine
106,Urdu,"Fiji,Pakistan"
107,Uzbek,"Afghanistan,Uzbekistan"


#### ANSWER TO QUESTION 3
- List all the countries and the total number of languages spoken.

In [26]:
QUERY = """SELECT name as Country, LENGTH(languageCode) - LENGTH(REPLACE(languageCode, ',','')) +1 as 'Lng Count' FROM countries"""

cursor.execute(QUERY)
b = cursor.fetchall()

ANSWER2 = pd.DataFrame(b, columns =['Country', 'Lng Count'])
ANSWER2

,Country,Lng Count
0,Afghanistan,3
1,Åland,1
2,Albania,1
3,Algeria,1
4,American Samoa,2
...,...,...
245,Wallis and Futuna,1
246,Western Sahara,1
247,Yemen,1
248,Zambia,1


In [17]:
if (cnx.is_connected()):
    cursor.close()
    cnx.close()
    print("MySQL connection is closed")

MySQL connection is closed


In [31]:
extract_and_transform_currency_data()

,code,countryIso2code
0,,AQ
1,AED,AE
2,AFN,AF
3,ALL,AL
4,AMD,AM
...,...,...
155,XOF,"BF,BJ,CI,GW,ML,NE,SN,TG"
156,XPF,"NC,PF,WF"
157,YER,YE
158,ZAR,ZA


In [ ]:
# id | name    
# 1  | a,b,c    
# 2  | b


# | name                | languageCode |

# >>>>>>

# select
#   countries.name,
#   SUBSTRING_INDEX(SUBSTRING_INDEX(countries.languageCode, ',', numbers.n), ',', -1) languageCode
# from
#   numbers inner join countries
#   on CHAR_LENGTH(countries.languageCode)
#      -CHAR_LENGTH(REPLACE(countries.languageCode, ',', ''))>=numbers.n-1
# order by
#   name, n